## Connecting to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ML_Project

/content/drive/MyDrive/ML_Project


## Imports

In [ ]:
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

## Reading Data

In [ ]:
df = pd.read_csv('SharedFiles/DataCSV/train.csv')
df = df.sample(frac = 1)  # to shuffe

In [ ]:
len(df)

967005

## Preprocessing Data

### Remove outliers: data with zeros

In [ ]:
df = df[(df['Blue'] != 0) | (df['Green'] != 0) | (df['Red'] != 0) | (df['NIR'] != 0)]
df

### Normalize data by dividing 10000

In [ ]:
df['Blue'] = df['Blue'] / 10000
df['Red'] = df['Red'] / 10000
df['Green'] = df['Green'] / 10000
df['NIR'] = df['NIR'] / 10000

df

### Calculate NDVI

In [ ]:
df['NDVI'] = (df["NIR"] - df['Red']) / (df["NIR"] + df['Red'])
df

## Train SVM

### Prepare train val split

In [ ]:
dfnumpy = df.to_numpy()
np.random.shuffle(dfnumpy)

X = dfnumpy[:, -5:]#.reshape(-1, 1)
Y = dfnumpy[:, 1]

In [ ]:
X_Train, X_Test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=0.3)

In [ ]:
print("X_Train.shape: ", X_Train.shape)
print("X_Test.shape: ", X_Test.shape)
print("y_train.shape: ", y_train.shape)
print("y_test.shape: ", y_test.shape)

### find best degree

In [ ]:
kfold=5
best_degree = 1
best_score = 0
scores = []

for i in range(1,10):
    svm_classifier = SVC(kernel="poly",degree=i)
    svmScores = cross_val_score(svm_classifier, X, Y.ravel(), cv=kfold, scoring="accuracy")
    score = statistics.mean(svmScores)
    scores.append(score)
    
    if score > best_score:
        best_score=score
        best_degree=i
    print('Average accuracy in 10- fold cross-validation for SVM with degree',i , ' is ', score)

print("\n\nBest degree is",best_degree," with score ",best_score)

### find optimum C

In [ ]:
scores = []
best_C = 1
best_score = 0
rangeC = 100
for c in range(1,rangeC):
    svm_classifier = SVC(kernel="poly",degree=best_degree,C=c/10)
    svmScores = cross_val_score(svm_classifier, X, Y.ravel(), cv=kfold, scoring="accuracy")
    score = statistics.mean(svmScores)
    scores.append(score)
    if score > best_score:
        best_score=score
        best_C=c/100
    #print('Average accuracy in 10- fold cross-validation for SVM with degree',best_degree ,"and C",c, ' is ', score)

print("\n\nBest C is",best_C," with score ",best_score)

### Train

In [ ]:
svm_classifier = SVC(kernel="poly",degree=best_degree,C=best_score)
svm_classifier.fit(X_Train,y_train)

Y_hat = svm_classifier.predict(X_Test)
print(classification_report(y_test,Y_hat))

## Train KNN

### Finding Best n_neighbors

In [ ]:
scores = [];

bestScore = 0;
bestK = 0;

for k in range(1, 35, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knnScores = cross_val_score(knn, X_Train, y_train, cv=5, scoring="accuracy")
    knnScore = statistics.mean(knnScores);
    scores.append(knnScore);
    
    print('Average accuracy in ', 5, '- fold cross-validation for KNN with K=', k, ' is ', knnScore)
    if (knnScore > bestScore):
        bestScore = knnScore;
        bestK = k

print('')
print('Best K is ', bestK, ' with best score as ', bestScore)

plt.plot(scores)
plt.title('K-fold Cross Validation, Test Accuracies with respect to K (for KNN)')

### Taining

In [ ]:
# Create the KNN classifier with k=1
clf = KNeighborsClassifier(n_neighbors=19)

# Use cross-validation to evaluate the model's accuracy
#scores = cross_val_score(clf, X, Y.ravel(), cv=5)
#acc = scores.mean()

# Fit the classifier to the data
clf.fit(X_Train, y_train)

# Making Prediction and Evaluation On Validation Set

In [ ]:
# Predict labels for new data
predictions = svm_classifier.predict(X_Test) # X_Test

In [ ]:
# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, predictions) # y_test
print(f'Accuracy: {accuracy:.2%}')

In [ ]:
# Compute the confusion matrix
labels = ['Tree cover', 'Shrubland', 'Grassland', 'Cropland', 'Built-up', 'Bare/sparse vegetation', 'Snow and ice','Permanent water bodies', 'Herbaceous wetland']

cm = confusion_matrix(y_test, predictions) # y_test
print(classification_report(y_test, predictions,target_names=labels)) # y_test
# print(cm)
cmd = ConfusionMatrixDisplay(cm, display_labels=labels)
cmd.plot()

# Final Test For Submission

## Reading Test Data

We are only normalizing data and calculating NDVI. When we delete outliers the results become not suitable for submission. Because number of rows changes 

In [ ]:
df_test = pd.read_csv('SharedFiles/DataCSV/test.csv')
len(df_test)

### Normalizing Data

In [ ]:
df_test['Blue'] = df_test['Blue'] / 10000
df_test['Red'] = df_test['Red'] / 10000
df_test['Green'] = df_test['Green'] / 10000
df_test['NIR'] = df_test['NIR'] / 10000

df_test

### Calculate NDVI

In [ ]:
df_test['NDVI'] = (df_test["NIR"] - df_test['Red']) / (df_test["NIR"] + df_test['Red'])
df_test['NDVI'] = df_test['NDVI'].fillna(0)

df_test

## Making Final Predictions To Write submission.csv

In [ ]:
dfnumpy = df_test.to_numpy()
X_test_submission = dfnumpy[:, -5:]#.reshape(-1, 1)

In [ ]:
# Predict labels for new data
predictions = clf.predict(X_test_submission)

In [ ]:
# Giving column name
df_result_submission = pd.DataFrame(predictions)
df_result_submission.columns=['Code']
df_result_submission.head()

In [ ]:
# Preparing Id column
df_result_submissionDeneme = df_result_submission
df_result_submissionDeneme['Code'] = df_result_submissionDeneme['Code'].astype(int)

df_result_submissionDeneme.insert(0, 'Id', range(0, 0 + len(df_result_submissionDeneme)))

df_result_submissionDeneme.shape
df_result_submissionDeneme.head()

In [ ]:
# Writing results without default index
df_result_submissionDeneme.to_csv('submission.csv',index=False)